<a href="https://colab.research.google.com/github/Vietchemistryyy/Fake_News_Detection_BERT/blob/main/notebooks/04_bert_traing_claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# CELL 1: Imports and Setup
# ============================================================================
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

from src.config import ModelConfig, TrainingConfig, MODELS_DIR, RESULTS_DIR
from src.dataset_claude import load_data_loaders
from src.model_claude import load_model, BERTFakeNewsClassifier
from src.train_claude import Trainer, train_model
from src.evaluate_claude import Evaluator
from src.utils import set_seed, Timer

print("✅ Imports successful!")

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

ModuleNotFoundError: No module named 'src'

In [ ]:
# ============================================================================
# CELL 2: Set Random Seed
# ============================================================================
print("\n" + "="*80)
print("SETTING RANDOM SEED")
print("="*80)

set_seed(TrainingConfig.SEED)
print(f"✅ Random seed set to: {TrainingConfig.SEED}")

In [ ]:
# ============================================================================
# CELL 3: Load Data
# ============================================================================
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

with Timer("Data loading"):
    train_loader, val_loader, test_loader = load_data_loaders(
        tokenizer_name=ModelConfig.MODEL_NAME
    )

print(f"\n✅ Data loaders created!")
print(f"   Training batches:   {len(train_loader)}")
print(f"   Validation batches: {len(val_loader)}")
print(f"   Test batches:       {len(test_loader)}")
print(f"   Batch size:         {ModelConfig.BATCH_SIZE}")

# Calculate total samples
total_train = len(train_loader) * ModelConfig.BATCH_SIZE
total_val = len(val_loader) * ModelConfig.BATCH_SIZE
total_test = len(test_loader) * ModelConfig.BATCH_SIZE

print(f"\n📊 Approximate sample counts:")
print(f"   Training:   ~{total_train:,}")
print(f"   Validation: ~{total_val:,}")
print(f"   Test:       ~{total_test:,}")


In [ ]:
# ============================================================================
# CELL 4: Test Data Loading
# ============================================================================
print("\n" + "="*80)
print("TESTING DATA LOADER")
print("="*80)

# Get one batch
batch = next(iter(train_loader))

print(f"\n📦 Batch structure:")
print(f"   Input IDs shape:      {batch['input_ids'].shape}")
print(f"   Attention mask shape: {batch['attention_mask'].shape}")
print(f"   Labels shape:         {batch['label'].shape}")

# Decode sample
tokenizer = AutoTokenizer.from_pretrained(ModelConfig.MODEL_NAME)
sample_text = tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True)
sample_label = batch['label'][0].item()

print(f"\n📝 Sample from batch:")
print(f"   Text: {sample_text[:200]}...")
print(f"   Label: {sample_label} ({'Fake' if sample_label == 1 else 'Real'})")

In [ ]:
# ============================================================================
# CELL 5: Initialize Model
# ============================================================================
print("\n" + "="*80)
print("INITIALIZING MODEL")
print("="*80)

with Timer("Model initialization"):
    model = load_model(
        model_name=ModelConfig.MODEL_NAME,
        num_labels=ModelConfig.NUM_LABELS,
        device=device
    )

print(f"\n✅ Model initialized!")
print(f"   Model: {ModelConfig.MODEL_NAME}")
print(f"   Num labels: {ModelConfig.NUM_LABELS}")
print(f"   Device: {device}")


In [ ]:
# ============================================================================
# CELL 6: Test Forward Pass
# ============================================================================
print("\n" + "="*80)
print("TESTING FORWARD PASS")
print("="*80)

model.eval()
with torch.no_grad():
    # Get batch
    batch = next(iter(train_loader))
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)

    # Forward pass
    logits = model(input_ids, attention_mask)

    # Get predictions
    probs = torch.softmax(logits, dim=1)
    predictions = torch.argmax(probs, dim=1)

print(f"✅ Forward pass successful!")
print(f"   Logits shape: {logits.shape}")
print(f"   Sample logits: {logits[0]}")
print(f"   Sample probs: {probs[0]}")
print(f"   Sample prediction: {predictions[0].item()}")
print(f"   Sample true label: {labels[0].item()}")


In [ ]:
# ============================================================================
# CELL 7: Initialize Trainer
# ============================================================================
print("\n" + "="*80)
print("INITIALIZING TRAINER")
print("="*80)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    learning_rate=ModelConfig.LEARNING_RATE,
    epochs=ModelConfig.NUM_EPOCHS
)

print(f"\n✅ Trainer initialized!")
print(f"   Optimizer: AdamW")
print(f"   Learning rate: {ModelConfig.LEARNING_RATE}")
print(f"   Epochs: {ModelConfig.NUM_EPOCHS}")
print(f"   Warmup steps: {ModelConfig.WARMUP_STEPS}")


In [ ]:
# ============================================================================
# CELL 8: Start Training
# ============================================================================
print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)
print("\n⚠️  This will take some time...")
print(f"   Estimated time per epoch: ~{len(train_loader) * 2 / 60:.1f} minutes")
print(f"   Total estimated time: ~{len(train_loader) * 2 * ModelConfig.NUM_EPOCHS / 60:.1f} minutes")
print("\n🚀 Training starting...\n")

# Train model
history = trainer.train(save_best=True, save_dir=MODELS_DIR)

print("\n✅ Training completed!")

In [ ]:
# ============================================================================
# CELL 9: Plot Training History
# ============================================================================
print("\n" + "="*80)
print("PLOTTING TRAINING HISTORY")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss
axes[0, 0].plot(history['train_loss'], label='Train Loss', marker='o')
axes[0, 0].plot(history['val_loss'], label='Val Loss', marker='s')
axes[0, 0].set_title('Loss over Epochs', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Accuracy
axes[0, 1].plot(history['train_acc'], label='Train Accuracy', marker='o')
axes[0, 1].plot(history['val_acc'], label='Val Accuracy', marker='s')
axes[0, 1].set_title('Accuracy over Epochs', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Learning Rate
axes[1, 0].plot(history['learning_rates'], marker='o', color='orange')
axes[1, 0].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Learning Rate')
axes[1, 0].grid(alpha=0.3)

# Comparison
epochs = range(1, len(history['train_loss']) + 1)
axes[1, 1].plot(epochs, history['train_acc'], label='Train Acc', marker='o', linestyle='--')
axes[1, 1].plot(epochs, history['val_acc'], label='Val Acc', marker='s', linestyle='--')
axes[1, 1].axhline(y=max(history['val_acc']), color='r', linestyle=':',
                   label=f'Best Val Acc: {max(history["val_acc"]):.4f}')
axes[1, 1].set_title('Training vs Validation Accuracy', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.suptitle('BERT Training History', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'visualizations' / 'training_history.png', dpi=300, bbox_inches='tight')
print("✅ Training history plot saved!")
plt.show()

# Print summary
print("\n📊 Training Summary:")
print(f"   Best validation loss: {min(history['val_loss']):.4f}")
print(f"   Best validation accuracy: {max(history['val_acc']):.4f}")
print(f"   Final train loss: {history['train_loss'][-1]:.4f}")
print(f"   Final train accuracy: {history['train_acc'][-1]:.4f}")

In [ ]:
# ============================================================================
# CELL 10: Load Best Model
# ============================================================================
print("\n" + "="*80)
print("LOADING BEST MODEL")
print("="*80)

from src.model import load_checkpoint

best_model_path = MODELS_DIR / ModelConfig.BEST_MODEL_NAME
best_model = load_checkpoint(best_model_path, device=device)

print(f"✅ Best model loaded from: {best_model_path}")

In [ ]:
# ============================================================================
# CELL 11: Evaluate on Test Set
# ============================================================================
print("\n" + "="*80)
print("EVALUATING ON TEST SET")
print("="*80)

evaluator = Evaluator(best_model, device=device)
test_metrics = evaluator.evaluate(test_loader, save_results=True)

print("\n📊 Test Set Results:")
print(f"   Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"   Precision: {test_metrics['precision']:.4f}")
print(f"   Recall:    {test_metrics['recall']:.4f}")
print(f"   F1-Score:  {test_metrics['f1_score']:.4f}")
print(f"   ROC-AUC:   {test_metrics['roc_auc']:.4f}")

In [ ]:
# ============================================================================
# CELL 12: Compare with Baseline
# ============================================================================
print("\n" + "="*80)
print("COMPARISON WITH BASELINE")
print("="*80)

# Load baseline results (if available)
baseline_metrics = {
    'accuracy': 0.85,   # Example - replace with actual baseline results
    'precision': 0.84,
    'recall': 0.86,
    'f1_score': 0.85
}

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'],
    'Baseline (TF-IDF + LR)': [
        baseline_metrics['accuracy'],
        baseline_metrics['precision'],
        baseline_metrics['recall'],
        baseline_metrics['f1_score'],
        0.90  # Approximate
    ],
    'BERT Model': [
        test_metrics['accuracy'],
        test_metrics['precision'],
        test_metrics['recall'],
        test_metrics['f1_score'],
        test_metrics['roc_auc']
    ]
})

comparison_df['Improvement'] = comparison_df['BERT Model'] - comparison_df['Baseline (TF-IDF + LR)']
comparison_df['Improvement %'] = (comparison_df['Improvement'] / comparison_df['Baseline (TF-IDF + LR)']) * 100

print("\n📊 Model Comparison:")
print(comparison_df.to_string(index=False))

# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(comparison_df))
width = 0.35

bars1 = ax.bar(x - width/2, comparison_df['Baseline (TF-IDF + LR)'],
              width, label='Baseline', color='skyblue')
bars2 = ax.bar(x + width/2, comparison_df['BERT Model'],
              width, label='BERT', color='salmon')

ax.set_xlabel('Metrics', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Baseline vs BERT Model Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Metric'])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{height:.3f}',
               ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'visualizations' / 'model_comparison.png', dpi=300, bbox_inches='tight')
print("\n✅ Comparison plot saved!")
plt.show()


In [ ]:
# ============================================================================
# CELL 13: Test Sample Predictions
# ============================================================================
print("\n" + "="*80)
print("TESTING SAMPLE PREDICTIONS")
print("="*80)

def predict_text(text, model, tokenizer, device):
    """Predict single text"""
    model.eval()

    # Tokenize
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=ModelConfig.MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Predict
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = torch.softmax(logits, dim=1)
        prediction = torch.argmax(probs, dim=1)

    return prediction.item(), probs[0].cpu().numpy()

# Test samples
test_samples = [
    "Breaking news: Scientists discover cure for cancer in major breakthrough",
    "You won't believe what happened next! Click here for shocking truth",
    "The president announced new economic policies in a press conference today",
    "Aliens landed in New York and nobody noticed! Government cover-up exposed"
]

print("\n🔍 Sample Predictions:\n")
for i, text in enumerate(test_samples, 1):
    pred, probs = predict_text(text, best_model, tokenizer, device)
    print(f"Sample {i}:")
    print(f"   Text: {text}")
    print(f"   Prediction: {'FAKE' if pred == 1 else 'REAL'}")
    print(f"   Confidence: Real={probs[0]:.3f}, Fake={probs[1]:.3f}")
    print()

In [ ]:
# ============================================================================
# CELL 14: Save Final Report
# ============================================================================
print("\n" + "="*80)
print("GENERATING FINAL REPORT")
print("="*80)

final_report = {
    "model_name": ModelConfig.MODEL_NAME,
    "training_config": {
        "epochs": ModelConfig.NUM_EPOCHS,
        "batch_size": ModelConfig.BATCH_SIZE,
        "learning_rate": ModelConfig.LEARNING_RATE,
        "max_length": ModelConfig.MAX_LENGTH
    },
    "training_history": {
        "best_val_loss": min(history['val_loss']),
        "best_val_acc": max(history['val_acc']),
        "final_train_loss": history['train_loss'][-1],
        "final_train_acc": history['train_acc'][-1]
    },
    "test_metrics": test_metrics,
    "comparison": comparison_df.to_dict()
}

from src.utils import save_json
report_path = RESULTS_DIR / 'metrics' / 'final_report.json'
save_json(final_report, report_path)

print(f"✅ Final report saved to: {report_path}")


In [ ]:
# ============================================================================
# CELL 15: Summary
# ============================================================================
print("\n" + "="*80)
print("🎉 TRAINING COMPLETE!")
print("="*80)

print("\n📌 Summary:")
print(f"   ✓ Model trained: {ModelConfig.MODEL_NAME}")
print(f"   ✓ Best validation accuracy: {max(history['val_acc']):.4f}")
print(f"   ✓ Test accuracy: {test_metrics['accuracy']:.4f}")
print(f"   ✓ Test F1-Score: {test_metrics['f1_score']:.4f}")
print(f"   ✓ Model saved: {MODELS_DIR / ModelConfig.BEST_MODEL_NAME}")

print("\n🎯 Next Steps:")
print("   1. Deploy model to production (Phase 3)")
print("   2. Build FastAPI backend")
print("   3. Create Next.js frontend")
print("   4. Set up MongoDB database")

print("\n" + "="*80)